In [1]:
from torchtext import data
import pandas as pd
import string
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
nlp = spacy.load('en')

In [2]:
TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField()

In [3]:
fields = [('text', TEXT), ('topic', LABEL)]

In [4]:
train_data, valid_data = data.TabularDataset.splits(
                                        path = 'data',
                                        train = 'train.csv',
                                        validation = 'val.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)

In [5]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.300d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [6]:
print(LABEL.vocab.stoi)

defaultdict(None, {'Bad Taste/Flavor': 0, 'Quality/Contaminated': 1, 'Not Effective': 2, 'Allergic': 3, 'Packaging': 4, 'Texture': 5, 'Shipment and delivery': 6, 'Customer Service': 7, 'Color and texture': 8, 'Too big to swallow': 9, 'Ingredients': 10, 'Expiry': 11, 'Smells Bad': 12, 'Pricing': 13, 'Wrong Product received': 14, 'Too Sweet': 15, 'Inferior to competitors': 16, 'False Advertisement': 17, "Didn't Like": 18, 'Customer Issues': 19, 'Hard to Chew': 20})


In [7]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data),
    sort_key=lambda x: len(x.text), #sort by s attribute (quote)
    sort_within_batch = True,
    batch_size=BATCH_SIZE,
    device=device)

In [8]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [9]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 170
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
WEIGHT_DECAY = 0.0001
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [10]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 2.1185, -0.4615,  0.6739,  ..., -0.6159,  0.0302, -0.0682],
        [-0.4318, -1.0575, -0.2638,  ...,  0.3831, -1.2745,  0.1936],
        [-0.1256,  0.0136,  0.1031,  ..., -0.3422, -0.0224,  0.1368],
        ...,
        [-1.2150, -1.0141, -0.5998,  ...,  0.3420, -0.0995,  0.3069],
        [-0.5372, -0.9808, -0.6435,  ...,  0.3492,  0.3977,  0.0348],
        [-0.3239,  1.6213, -0.7122,  ...,  0.1654, -0.4450, -0.0763]])

In [11]:
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [12]:
optimizer = optim.Adam(model.parameters(), weight_decay=WEIGHT_DECAY)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [13]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [14]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)
        
        loss = criterion(predictions, batch.topic)
        
        acc = categorical_accuracy(predictions, batch.topic)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [15]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.topic)
            
            acc = categorical_accuracy(predictions, batch.topic)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [16]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [17]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = train_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 2.383 | Train Acc: 29.86%
	 Val. Loss: 1.930 |  Val. Acc: 44.26%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 1.652 | Train Acc: 51.44%
	 Val. Loss: 1.475 |  Val. Acc: 54.31%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 1.288 | Train Acc: 60.93%
	 Val. Loss: 1.284 |  Val. Acc: 57.94%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 1.077 | Train Acc: 66.21%
	 Val. Loss: 1.255 |  Val. Acc: 55.87%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 0.965 | Train Acc: 68.11%
	 Val. Loss: 1.229 |  Val. Acc: 56.83%
Epoch: 06 | Epoch Time: 0m 1s
	Train Loss: 0.906 | Train Acc: 68.51%
	 Val. Loss: 1.205 |  Val. Acc: 56.45%
Epoch: 07 | Epoch Time: 0m 1s
	Train Loss: 0.845 | Train Acc: 69.06%
	 Val. Loss: 1.216 |  Val. Acc: 55.49%
Epoch: 08 | Epoch Time: 0m 1s
	Train Loss: 0.820 | Train Acc: 69.69%
	 Val. Loss: 1.227 |  Val. Acc: 54.64%
Epoch: 09 | Epoch Time: 0m 1s
	Train Loss: 0.792 | Train Acc: 70.27%
	 Val. Loss: 1.258 |  Val. Acc: 54.72%
Epoch: 10 | Epoch Time: 0m 1

In [18]:
model.load_state_dict(torch.load('model.pt'))

In [20]:
def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax()
    return max_preds.item()
    return tensor, length_tensor

In [21]:
predictions = []
df = pd.read_csv('./data/test.csv')
for i, x in enumerate(df.text_data.values):
    predictions.append(LABEL.vocab.itos[predict_class(model, x)])

In [1037]:
test_data_df = pd.read_csv('test.csv')
test_data_df['topic'] = predictions
test_data_df.to_csv('submission.csv', index=False)